# Cambiar Path absoluto a relativo para resolver path no encontrado en depth_metrics.csv

In [34]:
import csv
from pathlib import Path

# =========================
# CONFIG
# =========================
CSV_IN  = Path("Manzana/csv/depth_metrics.csv")
CSV_OUT = Path("Manzana/csv/depth_metrics_relative.csv")

# Prefijo absoluto a eliminar (Linux)
ABS_PREFIX = "/home/piper2/capturas_ros/Manzana/"

# =========================
# MAIN
# =========================
def main():
    if not CSV_IN.exists():
        raise FileNotFoundError(f"No existe: {CSV_IN}")

    CSV_OUT.parent.mkdir(parents=True, exist_ok=True)

    with open(CSV_IN, "r", newline="") as fin, open(CSV_OUT, "w", newline="") as fout:
        reader = csv.DictReader(fin)
        fieldnames = reader.fieldnames
        writer = csv.DictWriter(fout, fieldnames=fieldnames)
        writer.writeheader()

        n = 0
        for row in reader:
            fname = row.get("filename", "")

            if fname.startswith(ABS_PREFIX):
                row["filename"] = fname[len(ABS_PREFIX):]  # path relativo

            writer.writerow(row)
            n += 1

    print(f"CSV generado correctamente: {CSV_OUT}")
    print(f"Filas procesadas: {n}")

if __name__ == "__main__":
    main()


CSV generado correctamente: Manzana\csv\depth_metrics_relative.csv
Filas procesadas: 2000


# Filtrar Imagenes 10 > 1 --output: yolo_reduced

In [23]:
import os
import re
import csv
import shutil
from pathlib import Path

# =========================
# CONFIG
# =========================
INPUT_DIR = Path("Manzana")                  # asume que ejecutas dentro de ...\Manzana
IMAGES_DIR = INPUT_DIR / "images"
CSV_DIR    = INPUT_DIR / "csv"
DEPTH_METRICS = CSV_DIR / "depth_metrics_relative.csv"

YOLO_OUT = INPUT_DIR / "yolo_reduced"
YOLO_IMAGES = YOLO_OUT / "images"
YOLO_LABELS = YOLO_OUT / "labels"

N_PER_BLOCK = 10                        # 10 tomas por tópico
DEPTH_CHOICE_METRIC = "median_z_m"      # o "center_z_m"
MIN_VALID_PX = 1
COPY_MODE = "copy"                      # "copy" o "move"

REQUIRE_FISHEYE = True                  # si True: descarta bloque si no hay fisheye cercano
# Si quieres evitar el problema por fisheye faltante (1990/2000), pon False.

# Si quieres forzar umbral, pon por ejemplo int(2e9) para 2s; por defecto None (sin límite).
MAX_STAMP_DIFF_NS = None

# =========================
# HELPERS
# =========================
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def to_float(x):
    try:
        return float(x)
    except:
        return None

def to_int(x):
    try:
        return int(float(x))
    except:
        return 0

def parse_depth_row(row):
    # header esperado: stamp_ns, filename, center_z_m, min_z_m, median_z_m, valid_px
    return {
        "stamp_ns": row.get("stamp_ns", ""),
        "filename": row.get("filename", ""),
        "center_z_m": to_float(row.get("center_z_m", "")),
        "median_z_m": to_float(row.get("median_z_m", "")),
        "valid_px": to_int(row.get("valid_px", "0")),
    }

def extract_stamp_idx(nick: str, filename: str):
    """
    Espera: <nick>_<stamp>_<idx>.png
    Devuelve: (stamp:int, idx:str)
    """
    base = os.path.basename(filename)
    m = re.match(rf"^{re.escape(nick)}_(\d+)_(\d{{4}})\.png$", base)
    if not m:
        return None, None
    return int(m.group(1)), m.group(2)

def build_index(nick: str):
    """
    idx -> list[(stamp:int, path:Path)] ordenado por stamp
    """
    folder = IMAGES_DIR / nick
    idx_map = {}
    if not folder.exists():
        return idx_map

    for p in folder.glob(f"{nick}_*_????.png"):
        stamp, idx = extract_stamp_idx(nick, p.name)
        if stamp is None:
            continue
        idx_map.setdefault(idx, []).append((stamp, p))

    for idx in idx_map:
        idx_map[idx].sort(key=lambda t: t[0])
    return idx_map

def pick_closest(candidates, target_stamp, max_diff=None):
    if not candidates:
        return None
    best = min(candidates, key=lambda t: abs(t[0] - target_stamp))
    if max_diff is not None and abs(best[0] - target_stamp) > max_diff:
        return None
    return best  # (stamp, path)

def safe_copy_or_move(src: Path, dst: Path, mode="copy"):
    ensure_dir(dst.parent)
    if mode == "move":
        shutil.move(str(src), str(dst))
    else:
        shutil.copy2(str(src), str(dst))

# =========================
# MAIN
# =========================
def main():
    if not DEPTH_METRICS.exists():
        raise FileNotFoundError(f"No existe: {DEPTH_METRICS}")

    ensure_dir(YOLO_IMAGES)
    ensure_dir(YOLO_LABELS)

    idx_color = build_index("color")
    idx_fish  = build_index("fisheye")

    # Leer depth_metrics.csv
    rows = []
    with open(DEPTH_METRICS, "r", newline="") as f:
        reader = csv.DictReader(f)
        for r in reader:
            rows.append(parse_depth_row(r))

    # Filtrar depth válidas
    rows = [r for r in rows if r["valid_px"] >= MIN_VALID_PX and r["filename"]]

    # Orden por tiempo
    def sort_key(r):
        try:
            return int(r["stamp_ns"])
        except:
            return 0
    rows.sort(key=sort_key)

    kept = []
    for i in range(0, len(rows), N_PER_BLOCK):
        block = rows[i:i+N_PER_BLOCK]
        if not block:
            continue

        metric = DEPTH_CHOICE_METRIC
        def candidate_key(r):
            v = r.get(metric, None)
            vv = v if (v is not None) else float("-inf")
            return (vv, r["valid_px"])

        best = max(block, key=candidate_key)

        # Resolver path del depth: puede venir absoluto o relativo
        depth_path = Path(best["filename"])
        if not depth_path.exists():
            depth_path_try = INPUT_DIR / depth_path
            if depth_path_try.exists():
                depth_path = depth_path_try

        stamp_depth, idx = extract_stamp_idx("depth", depth_path.name)
        if stamp_depth is None:
            print(f"[SKIP] No pude parsear depth: {depth_path.name}")
            continue

        kept.append((stamp_depth, idx, depth_path))

    print(f"Bloques procesados: {len(kept)} (objetivo: {len(kept)} tríos).")

    n_ok = 0
    n_skip = 0
    n_partial = 0

    for stamp_depth, idx, depth_path in kept:
        best_color = pick_closest(idx_color.get(idx, []), stamp_depth, MAX_STAMP_DIFF_NS)
        best_fish  = pick_closest(idx_fish.get(idx, []),  stamp_depth, MAX_STAMP_DIFF_NS)

        if best_color is None:
            # sin color no sirve para YOLO típico
            n_skip += 1
            continue
        if REQUIRE_FISHEYE and best_fish is None:
            n_skip += 1
            continue

        base_id = f"{stamp_depth}_{idx}"

        dst_depth   = YOLO_IMAGES / f"depth_{base_id}.png"
        dst_color   = YOLO_IMAGES / f"color_{base_id}.png"

        safe_copy_or_move(depth_path, dst_depth, mode=COPY_MODE)
        safe_copy_or_move(best_color[1], dst_color, mode=COPY_MODE)

        if best_fish is not None:
            dst_fisheye = YOLO_IMAGES / f"fisheye_{base_id}.png"
            safe_copy_or_move(best_fish[1], dst_fisheye, mode=COPY_MODE)
        else:
            n_partial += 1

        # Labels placeholders (si entrenas solo con color, puedes borrar depth/fisheye labels)
        for prefix in ["depth", "color", "fisheye"]:
            label_path = YOLO_LABELS / f"{prefix}_{base_id}.txt"
            if not label_path.exists():
                label_path.write_text("")

        n_ok += 1

    print("Resumen:")
    print(f"  Copiados OK (con color): {n_ok}")
    print(f"  Parciales (sin fisheye): {n_partial}")
    print(f"  Bloques descartados:     {n_skip}")
    print(f"Salida YOLO: {YOLO_OUT.resolve()}")

if __name__ == "__main__":
    main()


Bloques procesados: 200 (objetivo: 200 tríos).
Resumen:
  Copiados OK (con color): 200
  Parciales (sin fisheye): 0
  Bloques descartados:     0
Salida YOLO: C:\Users\garci\OneDrive - UNIVERSIDAD ANDRES BELLO\Desktop\1.Universidad\PhdDISA\Tesis\vision\Ultralytics\Manzana\yolo_reduced


# Desde Matlab usar export_gtruth_for_python.m para generar gTruth_py_flat.mat para trabajar con el dataset con nuevo formato

In [ ]:
export_gtruth_for_python("gTruthXX.mat", "gTruth_py_flat.mat", "dropEmptyLabels", true);

# Filtrar imagenes no clases 

In [ ]:
C:/ProgramData/anaconda3/envs/yolo/python.exe filter_gtruth_flat_no_empty.py `
  --in_mat gTruth_py_flat.mat `
  --out_mat gTruth_py_flat_filtered.mat

# Crear labels en formato YOLO de gtruth.mat

Identificar qué labels NO se usan realmente


In [ ]:
C:/ProgramData/anaconda3/envs/yolo/python.exe gtruth_flat_to_yolo.py `
   --mat gTruth_py_flat_filtered.mat `
   --labels_out Manzana/yolo_reduced/labels `
   --yaml_out Manzana/yolo_reduced/data.yaml `
  --dataset_root "../"

# Division de Dataset en train y val

In [1]:
from pathlib import Path
import random
import shutil
import yaml
import numpy as np
from scipy.io import loadmat

# =========================
# CONFIG
# =========================
DATASET_DIR = Path("Manzana/yolo_reduced")
LABELS_DIR = DATASET_DIR / "labels"
IMAGES_DIR = DATASET_DIR / "images"
YAML_PATH = DATASET_DIR / "data.yaml"

# El .mat filtrado (con imageFiles ya limpios)
MAT_PATH = Path("gTruth_py_flat_filtered.mat")

# Split
VAL_RATIO = 0.2
SEED = 42

# Filtros
IMAGE_PREFIX = ""          # ej: "color_" si quieres solo RGB
IMAGE_EXTS = {".jpg", ".jpeg", ".png"}

# Modo: "move" o "copy"
# Recomendación: "copy" para no romper rutas originales
MODE = "copy"


# =========================
# HELPERS
# =========================
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)


def transfer(src: Path, dst: Path, mode: str):
    ensure_dir(dst.parent)
    if mode == "copy":
        shutil.copy2(src, dst)
    else:
        shutil.move(src, dst)


def as_list_str(x):
    x = np.atleast_1d(x)
    out = []
    for v in x.tolist():
        if isinstance(v, bytes):
            out.append(v.decode("utf-8", errors="ignore"))
        else:
            out.append(str(v))
    return out


def load_image_files_from_mat(mat_path: Path):
    m = loadmat(mat_path, squeeze_me=True, struct_as_record=False)
    if "imageFiles" not in m:
        raise RuntimeError(f"No existe 'imageFiles' en {mat_path}")
    image_files = as_list_str(m["imageFiles"])
    return [Path(p) for p in image_files]


def main():
    if not DATASET_DIR.exists():
        raise FileNotFoundError(f"No existe DATASET_DIR: {DATASET_DIR}")
    if not MAT_PATH.exists():
        raise FileNotFoundError(f"No existe MAT_PATH: {MAT_PATH}")
    if not LABELS_DIR.exists():
        raise FileNotFoundError(f"No existe LABELS_DIR: {LABELS_DIR}. Primero genera los .txt YOLO.")

    # subcarpetas destino
    img_train = IMAGES_DIR / "train"
    img_val   = IMAGES_DIR / "val"
    lab_train = LABELS_DIR / "train"
    lab_val   = LABELS_DIR / "val"
    for p in [img_train, img_val, lab_train, lab_val]:
        ensure_dir(p)

    # cargar paths originales
    originals = load_image_files_from_mat(MAT_PATH)

    # filtrar candidatos por prefijo/ext
    candidates = []
    missing_img = 0
    for p in originals:
        if p.suffix.lower() in IMAGE_EXTS and p.name.startswith(IMAGE_PREFIX):
            if p.is_file():
                candidates.append(p)
            else:
                missing_img += 1

    if not candidates:
        raise RuntimeError(
            f"No se encontraron imágenes existentes que cumplan prefijo '{IMAGE_PREFIX}' y extensiones {IMAGE_EXTS}.\n"
            f"Rutas faltantes (no existen en disco): {missing_img}"
        )

    # pares img/label válidos (label se busca en LABELS_DIR raíz)
    pairs = []
    missing_label = 0
    for img_path in candidates:
        label_path = LABELS_DIR / (img_path.stem + ".txt")
        if label_path.exists():
            pairs.append((img_path, label_path))
        else:
            missing_label += 1

    if not pairs:
        raise RuntimeError("No hay pares (imagen,label) válidos. Verifica nombres en labels/ y stems de imágenes.")

    print(f"Imágenes en .mat:                 {len(originals)}")
    print(f"Candidatas existentes (filtro):   {len(candidates)}")
    print(f"Pares (img,label) válidos:        {len(pairs)}")
    print(f"Rutas imagen faltantes:           {missing_img}")
    print(f"Sin label correspondiente:        {missing_label}")

    # split reproducible
    random.seed(SEED)
    random.shuffle(pairs)
    n_total = len(pairs)
    n_val = max(1, int(round(n_total * VAL_RATIO)))
    val_pairs = pairs[:n_val]
    train_pairs = pairs[n_val:]

    print(f"Train: {len(train_pairs)} | Val: {len(val_pairs)}")

    # transferir (desde origen original hacia estructura YOLO)
    for img_path, lab_path in train_pairs:
        transfer(img_path, img_train / img_path.name, MODE)
        transfer(lab_path, lab_train / lab_path.name, MODE)

    for img_path, lab_path in val_pairs:
        transfer(img_path, img_val / img_path.name, MODE)
        transfer(lab_path, lab_val / lab_path.name, MODE)

    # actualizar data.yaml
    if YAML_PATH.exists():
        with open(YAML_PATH, "r", encoding="utf-8") as f:
            data = yaml.safe_load(f) or {}
    else:
        data = {}

    data["path"] = str(DATASET_DIR).replace("\\", "/")
    data["train"] = "images/train"
    data["val"]   = "images/val"

    with open(YAML_PATH, "w", encoding="utf-8") as f:
        yaml.safe_dump(data, f, sort_keys=False, allow_unicode=True)

    print(f"OK. data.yaml actualizado: {YAML_PATH}")
    print("Estructura final:")
    print(f"  {img_train}")
    print(f"  {img_val}")
    print(f"  {lab_train}")
    print(f"  {lab_val}")


if __name__ == "__main__":
    main()


Imágenes en .mat:                 1258
Candidatas existentes (filtro):   1258
Pares (img,label) válidos:        1258
Rutas imagen faltantes:           0
Sin label correspondiente:        0
Train: 1006 | Val: 252
OK. data.yaml actualizado: Manzana\yolo_reduced\data.yaml
Estructura final:
  Manzana\yolo_reduced\images\train
  Manzana\yolo_reduced\images\val
  Manzana\yolo_reduced\labels\train
  Manzana\yolo_reduced\labels\val


# Entrenamiento y Validacion YOLO11

In [3]:
from pathlib import Path
import shutil

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

def is_label_empty(label_path: Path) -> bool:
    """True si el .txt no contiene ninguna anotación válida."""
    if not label_path.exists():
        # Si no existe label, en práctica también es 'vacío' para YOLO
        return True

    txt = label_path.read_text(encoding="utf-8", errors="ignore").strip()
    if not txt:
        return True

    valid_lines = 0
    for line in txt.splitlines():
        line = line.strip()
        if not line:
            continue
        parts = line.split()
        # YOLO seg suele tener: cls x y w h ... (y luego coords de máscara)
        # YOLO det: cls x y w h
        # Consideramos válida si al menos tiene 5 tokens y cls es int.
        if len(parts) < 5:
            continue
        try:
            int(parts[0])
            # no validamos floats estrictamente para no romper formatos
            valid_lines += 1
        except ValueError:
            continue

    return valid_lines == 0

def find_image_for_label(label_path: Path, images_dir: Path) -> Path | None:
    """Dado labels/xxx/name.txt busca images/xxx/name.(jpg|png|...)"""
    stem = label_path.stem
    for ext in IMG_EXTS:
        candidate = images_dir / f"{stem}{ext}"
        if candidate.exists():
            return candidate
    return None

def quarantine_split(images_dir: str, labels_dir: str, quarantine_root: str):
    images_dir = Path(images_dir)
    labels_dir = Path(labels_dir)
    quarantine_root = Path(quarantine_root)

    q_images = quarantine_root / "images" / images_dir.name   # train o val
    q_labels = quarantine_root / "labels" / labels_dir.name   # train o val
    q_images.mkdir(parents=True, exist_ok=True)
    q_labels.mkdir(parents=True, exist_ok=True)

    empty_labels = []
    missing_images = []

    for label_path in labels_dir.glob("*.txt"):
        if is_label_empty(label_path):
            empty_labels.append(label_path)
            img_path = find_image_for_label(label_path, images_dir)
            if img_path is None:
                missing_images.append(label_path)
                # igual movemos el label vacío
                shutil.move(str(label_path), str(q_labels / label_path.name))
                continue

            # mover imagen y label
            shutil.move(str(img_path), str(q_images / img_path.name))
            shutil.move(str(label_path), str(q_labels / label_path.name))

    return empty_labels, missing_images

# ========= AJUSTA ESTAS RUTAS A TU DATASET =========
DATA_ROOT = Path(r"C:\Users\garci\OneDrive - UNIVERSIDAD ANDRES BELLO\Desktop\1.Universidad\PhdDISA\Tesis\vision\Ultralytics\Manzana\yolo_reduced")

train_images = DATA_ROOT / "images" / "train"
train_labels = DATA_ROOT / "labels" / "train"
val_images   = DATA_ROOT / "images" / "val"
val_labels   = DATA_ROOT / "labels" / "val"

quarantine_root = DATA_ROOT / "quarantine_empty_labels"

# ========= EJECUCIÓN =========
train_empty, train_missing = quarantine_split(train_images, train_labels, quarantine_root)
val_empty, val_missing     = quarantine_split(val_images, val_labels, quarantine_root)

print(f"[TRAIN] Labels vacíos movidos: {len(train_empty)} | Labels sin imagen: {len(train_missing)}")
print(f"[VAL]   Labels vacíos movidos: {len(val_empty)} | Labels sin imagen: {len(val_missing)}")
print(f"Quarantine en: {quarantine_root}")


[TRAIN] Labels vacíos movidos: 0 | Labels sin imagen: 0
[VAL]   Labels vacíos movidos: 0 | Labels sin imagen: 0
Quarantine en: C:\Users\garci\OneDrive - UNIVERSIDAD ANDRES BELLO\Desktop\1.Universidad\PhdDISA\Tesis\vision\Ultralytics\Manzana\yolo_reduced\quarantine_empty_labels


In [3]:
%pip install -U ultralytics
from ultralytics import YOLO
from pathlib import Path

DATASET_YAML = Path("Manzana/yolo_reduced/data.yaml")
PROJECT_DIR  = Path("Manzana")
RUN_NAME     = "V12s_640"

# Modelo base YOLO11 segmentation (elige tamaño: n, s, m, l, x)
# Ejemplos comunes:
#   yolo11n-seg.pt  (rápido)
#   yolo11s-seg.pt  (mejor calidad)
model = YOLO("yolo26s-seg.pt")

model.train(
    data=str(DATASET_YAML),
    imgsz=640,
    epochs=100,
    batch=8,          # ajusta según GPU/VRAM
    device=0,         # 0 si tienes GPU; "cpu" si no
    project=str(PROJECT_DIR),
    name=RUN_NAME,
    exist_ok=True,
    workers=8,
    # opcional: si tu dataset es pequeño, ayuda:
    # patience=30,
    # augment=True,
)

# Evaluación final sobre val
metrics = model.val(data=str(DATASET_YAML), device=0)
print(metrics)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Ultralytics 8.4.8  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Manzana\yolo_reduced\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26s-seg.p

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
from pathlib import Path

# === CONFIG ===
MODEL_PATH = r"runs\segment\Manzana\V12s_640\weights\best.pt"
SRC_DIR    = Path(r"Manzana\yolo_reduced\images\val")
OUT_DIR    = Path(r"Manzana\V5\Imagenes_Poster")
OUT_DIR.mkdir(parents=True, exist_ok=True)

CONF = 0.25
ALPHA = 0.45
DRAW_BOXES = False
DRAW_LABELS = True

model = YOLO(MODEL_PATH)

palette = [
    (255,  99,  71),
    ( 30, 144, 255),
    ( 50, 205,  50),
    (238, 130, 238),
    (255, 215,   0),
    (  0, 206, 209),
]

def clamp_color(c):
    return palette[c % len(palette)]

img_paths = sorted([p for p in SRC_DIR.glob("*") if p.suffix.lower() in [".jpg", ".jpeg", ".png"]])

saved = 0
skipped = 0

for p in img_paths:
    img_bgr = cv2.imread(str(p))
    if img_bgr is None:
        skipped += 1
        continue

    res = model.predict(source=img_bgr, conf=CONF, verbose=False)[0]
    out = img_bgr.copy()
    H, W = out.shape[:2]

    # --- Dibujar máscaras (si hay) ---
    if res.masks is not None and res.boxes is not None and len(res.boxes) > 0:
        masks = res.masks.data.cpu().numpy()  # (N,h,w)
        clss  = res.boxes.cls.cpu().numpy().astype(int)
        confs = res.boxes.conf.cpu().numpy()

        overlay = np.zeros((H, W, 3), dtype=np.uint8)

        for i in range(masks.shape[0]):
            color = clamp_color(clss[i])

            m = masks[i].astype(np.uint8)
            if m.shape[0] != H or m.shape[1] != W:
                m = cv2.resize(m, (W, H), interpolation=cv2.INTER_NEAREST)

            m = m.astype(bool)
            overlay[m] = color

            if DRAW_BOXES:
                x1, y1, x2, y2 = res.boxes.xyxy[i].cpu().numpy().astype(int)
                cv2.rectangle(out, (x1, y1), (x2, y2), color, 2)

            if DRAW_LABELS:
                name = model.names[clss[i]]
                txt = f"{name} {confs[i]:.2f}"
                x1, y1, x2, y2 = res.boxes.xyxy[i].cpu().numpy().astype(int)
                tx, ty = x1, max(0, y1 - 6)
                cv2.putText(out, txt, (tx, ty),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2, cv2.LINE_AA)

        out = cv2.addWeighted(out, 1.0, overlay, ALPHA, 0)

    # Guardar SIEMPRE (aunque no haya detecciones, guarda la imagen original)
    cv2.imwrite(str(OUT_DIR / f"{p.stem}_overlay.png"), out)
    saved += 1

print(f"Listo. Overlays en: {OUT_DIR}")
print(f"Imágenes encontradas: {len(img_paths)} | Guardadas: {saved} | Saltadas (cv2.imread falló): {skipped}")


Listo. Overlays en: Manzana\V5\Imagenes_Poster
Imágenes encontradas: 491 | Guardadas: 491 | Saltadas (cv2.imread falló): 0


In [ ]:
%pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="9AymnScbVJYmCAvctAyJ")
project = rf.workspace("b6410025ws-m7yuo").project("fruit-segmentation-w3zab")
version = project.version(1)
dataset = version.download("yolov11")
                

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 68.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 3.0 MB/s  0:00:01 eta 0:00:010m
  Attempting uninstall: idna╺━━━━━━━━━━━━━━━━━━━ 4/8 [opencv-python-headless]
    Found existing installation: idna 3.11m━━━━━━━━━━━━━━━━━━━ 4/8 [opencv-python-headless]
    Uninstalling idna-3.11:m╺━━━━━━━━━━━━━━━━━━━ 4/8 [opencv-python-headless]
      Successfully uninstalled idna-3.1190m━━━━━━━━━━━━━━━━━━━ 4/8 [opencv-python-headless]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [roboflow]5/8 [idna]thon-headless]
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to fruit-Segmentation-1 in yolov11:: 100%|██████████| 196/196 [00:00<00:00, 10311.49it/s]


In [4]:
%pip install pathlib
%pip install pandas
%pip install matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# ========= CONFIG =========
CSV_YOLO11 = r"runs\segment\Manzana\V11s_640\results_V11s_640.csv"   # <- cambia a tu ruta
CSV_YOLO26 = r"runs\segment\Manzana\V26s_640\results_V26s_640.csv"   # <- cambia a tu ruta
OUT_DIR = Path("figures_quant")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Métricas que quieres graficar (puedes editar esta lista)
METRICS = [
    "metrics/mAP50-95(B)",
    "metrics/mAP50-95(M)",
    "metrics/recall(M)",
]

# ========= HELPERS =========
def load_last_metrics(csv_path: str, model_name: str) -> dict:
    df = pd.read_csv(csv_path)

    # Normalizar nombres de columnas (por si hay espacios raros)
    df.columns = [c.strip() for c in df.columns]

    # Tomar la última época registrada
    last = df.iloc[-1]

    out = {"model": model_name}
    for k in METRICS:
        if k not in df.columns:
            raise KeyError(
                f"No encontré la columna '{k}' en {csv_path}.\n"
                f"Columnas disponibles (primeras 30): {list(df.columns)[:30]}"
            )
        out[k] = float(last[k])
    return out, df

def pretty_label(metric_key: str) -> str:
    # Etiquetas más lindas para el paper
    mapping = {
        "metrics/mAP50-95(B)": "mAP50–95 (Boxes)",
        "metrics/mAP50-95(M)": "mAP50–95 (Masks)",
        "metrics/recall(M)":   "Recall (Masks)",
        "metrics/recall(B)":   "Recall (Boxes)",
        "metrics/precision(M)":"Precision (Masks)",
        "metrics/precision(B)":"Precision (Boxes)",
        "metrics/mAP50(B)":    "mAP50 (Boxes)",
        "metrics/mAP50(M)":    "mAP50 (Masks)",
    }
    return mapping.get(metric_key, metric_key)

# ========= MAIN =========
m11, df11 = load_last_metrics(CSV_YOLO11, "YOLO11-seg")
m26, df26 = load_last_metrics(CSV_YOLO26, "YOLO26-seg")

# --- Bar chart comparativo (última época) ---
labels = [pretty_label(k) for k in METRICS]
x = range(len(METRICS))

y11 = [m11[k] for k in METRICS]
y26 = [m26[k] for k in METRICS]

plt.figure(figsize=(7.2, 3.8))
bar_w = 0.38

plt.bar([i - bar_w/2 for i in x], y11, width=bar_w, label=m11["model"])
plt.bar([i + bar_w/2 for i in x], y26, width=bar_w, label=m26["model"])

plt.xticks(list(x), labels, rotation=0)
plt.ylim(0, 1.0)
plt.ylabel("Score")
plt.legend()
plt.tight_layout()

out_png = OUT_DIR / "quant_comparison_bar.png"
out_pdf = OUT_DIR / "quant_comparison_bar.pdf"
plt.savefig(out_png, dpi=300)
plt.savefig(out_pdf)  # PDF sirve mucho para papers
plt.close()

print("✅ Guardado:", out_png)
print("✅ Guardado:", out_pdf)

# --- (OPCIONAL) Curvas por época para appendix o análisis ---
# Descomenta si te interesa tener curvas:
"""
CURVE_METRICS = ["metrics/mAP50-95(M)", "metrics/mAP50-95(B)", "metrics/recall(M)"]

def plot_curves(df, title, out_path):
    plt.figure(figsize=(7.2, 3.8))
    for k in CURVE_METRICS:
        if k in df.columns:
            plt.plot(df[k].values, label=pretty_label(k))
    plt.ylim(0, 1.0)
    plt.xlabel("Epoch")
    plt.ylabel("Score")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_path, dpi=300)
    plt.close()

plot_curves(df11, "YOLO11-seg validation curves", OUT_DIR / "yolo11_curves.png")
plot_curves(df26, "YOLO26-seg validation curves", OUT_DIR / "yolo26_curves.png")
print("✅ Curvas guardadas en", OUT_DIR)
"""


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------  9.7/9.9 MB 86.6 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 77.0 MB/s  0:00:00
Using cached tzdata-2025.3-py2.py3-none-any.whl (348 kB)

   ---------------------------------------- 0/2 [tzdata]
   ---------------------------------------- 0/2 [tzdata]
   ---------------------------------------- 0/2 [tzdata]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ------------------- 1/2 [pandas]
   -------------------- ---------

'\nCURVE_METRICS = ["metrics/mAP50-95(M)", "metrics/mAP50-95(B)", "metrics/recall(M)"]\n\ndef plot_curves(df, title, out_path):\n    plt.figure(figsize=(7.2, 3.8))\n    for k in CURVE_METRICS:\n        if k in df.columns:\n            plt.plot(df[k].values, label=pretty_label(k))\n    plt.ylim(0, 1.0)\n    plt.xlabel("Epoch")\n    plt.ylabel("Score")\n    plt.title(title)\n    plt.legend()\n    plt.tight_layout()\n    plt.savefig(out_path, dpi=300)\n    plt.close()\n\nplot_curves(df11, "YOLO11-seg validation curves", OUT_DIR / "yolo11_curves.png")\nplot_curves(df26, "YOLO26-seg validation curves", OUT_DIR / "yolo26_curves.png")\nprint("✅ Curvas guardadas en", OUT_DIR)\n'